In [ ]:
import openai
from openai import OpenAI
client = OpenAI("api_key"="")

In [ ]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{ "role":"user", "content":"is it too late to join the course" }]
)

In [ ]:
import os
os.environ